In [1]:
library(tidyverse)
library(broom)

── Attaching packages ─────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



# Load data files

In [2]:
covar_df <- read_tsv('../data/phenotypes/gwas_covariates.covar', col_types = cols())

covar_df %>% nrow

covar_df %>% head(0)

[1] 337147

FID,IID,sex,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,age_squared,age_sex,age_squared_sex
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


In [3]:
phenotypes_df <- read_tsv('../data/saved_qtphenproxy_I63/predictions.pheno', col_types = cols())

phenotypes_df %>% nrow

phenotypes_df %>% head(0)

[1] 353252

FID,IID,target,qtphenproxy,qtphenproxy_h2
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


In [4]:
genotypes_df <- read_tsv('../data/genotypes/marker_genotypes.raw', col_types = cols())

genotypes_df %>% nrow

genotypes_df %>% head(0)

[1] 487409

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


FID,IID,PAT,MAT,SEX,PHENOTYPE,rs6695915_A,rs138364069_C,rs3176471_A,rs11206510_T,⋯,rs2632512_C,rs7342953_C,rs9899728_G,rs7506045_C,rs2229383_T,rs1122608_G,rs8103309_T,rs55791371_A,rs16964543_T,rs56131196_G
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


# Run regressions

In [5]:
gwas_data_df <- genotypes_df %>% 
    inner_join(phenotypes_df, by = c('FID', 'IID')) %>%
    inner_join(covar_df, by = c('FID', 'IID'))

gwas_data_df %>% nrow

gwas_data_df %>% head(0)

[1] 237101

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


FID,IID,PAT,MAT,SEX,PHENOTYPE,rs6695915_A,rs138364069_C,rs3176471_A,rs11206510_T,⋯,PC4,PC5,PC6,PC7,PC8,PC9,PC10,age_squared,age_sex,age_squared_sex
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


In [6]:
snp_ids <- genotypes_df %>% select(-FID, -IID, -PAT, -MAT, -SEX, -PHENOTYPE) %>% colnames

snp_ids %>% length

[1] 103

In [7]:
gwas_results_df <- data.frame()

for (outcome in c('target', 'qtphenproxy', 'qtphenproxy_h2')) {
    if (outcome == 'target') {
        family = binomial
    } else {
        family = gaussian
    }
    
    for (snp_id in snp_ids) {
        reg_form <- paste(outcome, '~', 
                          paste(snp_id, 'sex', 'age', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 
                                        'PC8', 'PC9', 'PC10', 'age_squared', 'age_sex', 'age_squared_sex', sep='+'))
        reg_rows <- glm(reg_form, data = gwas_data_df, family = family) %>% 
            tidy %>%
            mutate(outcome = outcome, snp_id = snp_id)
        
        gwas_results_df <- bind_rows(gwas_results_df, reg_rows)
    }
}

gwas_results_df %>% write_tsv("../data/gwas_results/I63.tsv")

gwas_results_df %>% head(2)

,term,estimate,std.error,statistic,p.value,outcome,snp_id
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,(Intercept),-10.02454866,4.83634968,-2.0727510,0.03819546,target,rs6695915_A
2,rs6695915_A,-0.02402496,0.04700061,-0.5111628,0.60923706,target,rs6695915_A


In [8]:
gwas_results_df %>%
    filter(str_detect(term, 'rs')) %>%
    mutate(significant = p.value < 0.05) %>%
    group_by(outcome) %>%
    summarize(n_significant = sum(significant), .groups = 'drop')

outcome,n_significant
<chr>,<int>
qtphenproxy,6
qtphenproxy_h2,20
target,19
